In [1]:
# check if import is working
import gnn_tracking
# check that we have a GPU
import torch
# should show True
torch.cuda.is_available()

True

In [2]:
!nvidia-smi

Thu Jun 29 03:51:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   30C    P0    57W / 500W |      3MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import pickle
from pathlib import Path

from gnn_tracking.models.graph_construction import GraphConstructionFCNN
from tcn_trainer_mdmm import TCNTrainer

from gnn_tracking.metrics.losses import GraphConstructionHingeEmbeddingLoss
from gnn_tracking.utils.loading import get_loaders, TrackingDataset

In [5]:
graph_dir = Path("/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1")
assert graph_dir.is_dir()

In [12]:
dirs = []
for file in os.listdir(graph_dir):
    d = os.path.join(graph_dir, file)
    if os.path.isdir(d):
        dirs.append(d)

In [13]:
dirs

['/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_6',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_2',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_8',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_4',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_5',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_1',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_7',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_3',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_9',
 '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_0']

In [15]:
datasets = {
    "train": TrackingDataset(dirs, stop=810),
    "val": TrackingDataset(dirs, start=810, stop=900),
}
loaders = get_loaders(datasets, batch_size=4)

[04:15:50] INFO: DataLoader will load 810 graphs (out of 900 available).
[04:15:50] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_0/data21025_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_8/data21999_s0.pt
[04:15:50] INFO: DataLoader will load 90 graphs (out of 900 available).
[04:15:50] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_9/data21009_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v5/part1_pt1/part_1_9/data21986_s0.pt
[04:15:50] DEBUG: Parameters for data loader 'train': {'batch_size': 4, 'num_workers': 1, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x1529e6474310>, 'pin_memory': True, 'shuffle': None}
[04:15:50] DEBUG: Parameters for data loader 'val': {'batch_size': 1, 'num_workers': 1, 'sampler': None, 'pin_memory': True, 'shuf

In [16]:
def list_of_dicts_to_dict_of_lists(list_of_dicts):
    dict_of_lists = {}
    for dictionary in list_of_dicts:
        for key, value in dictionary.items():
            dict_of_lists.setdefault(key, []).append(value)
    return dict_of_lists

In [17]:
def train_model_mdmm(epsilon,
                     damping,
                     main_weight,
                     constraint_weight,
                     loaders,
                     main_loss="attractive",
                     constraint_loss="repulsive",
                     num_epochs=50
                    ):
    
    main_loss_functions = {
        "embedding_loss": (GraphConstructionHingeEmbeddingLoss(), {main_loss: main_weight}),
    }
    constraint_loss_functions = {
        "embedding_loss": (GraphConstructionHingeEmbeddingLoss(), {constraint_loss: (constraint_weight, epsilon, damping)}),
    }

    model = GraphConstructionFCNN(
        in_dim = 14,
        hidden_dim = 64,
        out_dim = 10,
        depth = 4,
        beta = 0.4
    )

    trainer = TCNTrainer(
        model=model,
        loaders=loaders,
        main_loss_functions=main_loss_functions,
        constraint_loss_functions=constraint_loss_functions,
        lr=0.005,
    )

    loss_history = trainer.train(epochs=num_epochs)
    return list_of_dicts_to_dict_of_lists(loss_history)

In [22]:
def save_loss_history(constraints, path, main_loss="attractive", constraint_loss="repulsive"):
    if not os.path.exists(path):
        os.makedirs(path)
    
    for constraint in constraints:
        print(f'Training for = {constraint}')
        loss_history = train_model_mdmm(epsilon=constraint[0],
                                damping=constraint[1],
                                main_weight=constraint[2],
                                constraint_weight=constraint[3],
                                loaders=loaders,
                                main_loss=main_loss,
                                constraint_loss=constraint_loss,
                                num_epochs=10)
        model_dict = {'loss_history':loss_history,
                      'epsilon':constraint[0],
                      'damping':constraint[1],
                      'weight':constraint[2]}
        
        file_path = os.path.join(path, f'{constraint[0]}_{constraint[1]}_{constraint[2]}_{constraint[3]}.pkl')

        with open(file_path, "wb") as f:
            pickle.dump(model_dict,f)
            f.close()
        print("\n")

In [19]:
constraints_1 = [(0.0009653154573041118, 10.0, 1.0, 1.0),
               (0.0004771597001106582, 10.0, 1.0, 1.0),
               (0.00028893887271952276, 10.0, 1.0, 1.0),
               (0.00016839923191582784, 10.0, 1.0, 1.0),
               (0.00010062895363475553, 10.0, 1.0, 1.0)]

In [20]:
!pwd

/home/dc3896/notebooks


In [21]:
save_loss_history(constraints_1, "loss_histories/damping_10")

[04:18:17 TCNTrainer] INFO: Using device cuda


Training for scaling coefficients = (0.0009653154573041118, 10.0, 1.0, 1.0)


/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:18:21 TCNTrainer] DEBUG: Epoch 1 (    0/203): Total=   0.03089, embedding_loss_attractive=   0.03089, embedding_loss_repulsive=   0.00124
[04:18:23 TCNTrainer] DEBUG: Epoch 1 (   10/203): Total=   0.00429, embedding_loss_attractive=   0.00404, embedding_loss_repulsive=   0.00471
[04:18:23 TCNTrainer] DEBUG: Epoch 1 (   20/203): Total=   0.00317, embedding_loss_attractive=   0.00287, embedding_loss_repulsive=   0.00372
[04:18:23 TCNTrainer] DEBUG: Epoch 1 (   30/203): Total=   0.00288, embedding_loss_attractive=   0.00226, embedding_loss_repulsive=   0.

[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  120/203): Total=   0.00189, embedding_loss_attractive=   0.00193, embedding_loss_repulsive=   0.00091
[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  130/203): Total=   0.00186, embedding_loss_attractive=   0.00184, embedding_loss_repulsive=   0.00100
[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  140/203): Total=   0.00174, embedding_loss_attractive=   0.00166, embedding_loss_repulsive=   0.00109
[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  150/203): Total=   0.00177, embedding_loss_attractive=   0.00169, embedding_loss_repulsive=   0.00109
[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  160/203): Total=   0.00168, embedding_loss_attractive=   0.00159, embedding_loss_repulsive=   0.00111
[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  170/203): Total=   0.00173, embedding_loss_attractive=   0.00170, embedding_loss_repulsive=   0.00101
[04:18:30 TCNTrainer] DEBUG: Epoch 2 (  180/203): Total=   0.00177, embedding_loss_attractive=   0.00177, embedding_loss_repulsive=   0.00096
[04:18

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:18:36 TCNTrainer] DEBUG: Epoch 4 (    0/203): Total=   0.00153, embedding_loss_attractive=   0.00157, embedding_loss_repulsive=   0.00091
[04:18:36 TCNTrainer] DEBUG: Epoch 4 (   10/203): Total=   0.00155, embedding_loss_attractive=   0.00156, embedding_loss_repulsive=   0.00095
[04:18:36 TCNTrainer] DEBUG: Epoch 4 (   20/203): Total=   0.00148, embedding_loss_attractive=   0.00159, embedding_loss_repulsive=   0.00080
[04:18:36 TCNTrainer] DEBUG: Epoch 4 (   30/203): Total=   0.00161, embedding_loss_attractive=   0.00154, embedding_loss_repulsive=   0.

...

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:21:26 TCNTrainer] DEBUG: Epoch 4 (    0/203): Total=   0.00269, embedding_loss_attractive=   0.00204, embedding_loss_repulsive=   0.00056
[04:21:26 TCNTrainer] DEBUG: Epoch 4 (   10/203): Total=   0.00275, embedding_loss_attractive=   0.00212, embedding_loss_repulsive=   0.00054
[04:21:26 TCNTrainer] DEBUG: Epoch 4 (   20/203): Total=   0.00242, embedding_loss_attractive=   0.00197, embedding_loss_repulsive=   0.00042
[04:21:26 TCNTrainer] DEBUG: Epoch 4 (   30/203): Total=   0.00260, embedding_loss_attractive=   0.00209, embedding_loss_repulsive=   0.

[04:21:32 TCNTrainer] DEBUG: Epoch 5 (  120/203): Total=   0.00269, embedding_loss_attractive=   0.00221, embedding_loss_repulsive=   0.00036
[04:21:32 TCNTrainer] DEBUG: Epoch 5 (  130/203): Total=   0.00269, embedding_loss_attractive=   0.00215, embedding_loss_repulsive=   0.00039
[04:21:32 TCNTrainer] DEBUG: Epoch 5 (  140/203): Total=   0.00270, embedding_loss_attractive=   0.00218, embedding_loss_repulsive=   0.00037
[04:21:32 TCNTrainer] DEBUG: Epoch 5 (  150/203): Total=   0.00259, embedding_loss_attractive=   0.00201, embedding_loss_repulsive=   0.00040
[04:21:33 TCNTrainer] DEBUG: Epoch 5 (  160/203): Total=   0.00270, embedding_loss_attractive=   0.00214, embedding_loss_repulsive=   0.00039
[04:21:33 TCNTrainer] DEBUG: Epoch 5 (  170/203): Total=   0.00269, embedding_loss_attractive=   0.00215, embedding_loss_repulsive=   0.00038
[04:21:33 TCNTrainer] DEBUG: Epoch 5 (  180/203): Total=   0.00269, embedding_loss_attractive=   0.00210, embedding_loss_repulsive=   0.00040
[04:21

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:21:39 TCNTrainer] DEBUG: Epoch 7 (    0/203): Total=   0.00265, embedding_loss_attractive=   0.00212, embedding_loss_repulsive=   0.00033
[04:21:39 TCNTrainer] DEBUG: Epoch 7 (   10/203): Total=   0.00265, embedding_loss_attractive=   0.00213, embedding_loss_repulsive=   0.00033
[04:21:39 TCNTrainer] DEBUG: Epoch 7 (   20/203): Total=   0.00256, embedding_loss_attractive=   0.00214, embedding_loss_repulsive=   0.00028
[04:21:39 TCNTrainer] DEBUG: Epoch 7 (   30/203): Total=   0.00268, embedding_loss_attractive=   0.00213, embedding_loss_repulsive=   0.

[04:21:45 TCNTrainer] DEBUG: Epoch 8 (  120/203): Total=   0.00278, embedding_loss_attractive=   0.00235, embedding_loss_repulsive=   0.00026
[04:21:45 TCNTrainer] DEBUG: Epoch 8 (  130/203): Total=   0.00273, embedding_loss_attractive=   0.00216, embedding_loss_repulsive=   0.00031
[04:21:45 TCNTrainer] DEBUG: Epoch 8 (  140/203): Total=   0.00268, embedding_loss_attractive=   0.00228, embedding_loss_repulsive=   0.00025
[04:21:45 TCNTrainer] DEBUG: Epoch 8 (  150/203): Total=   0.00277, embedding_loss_attractive=   0.00221, embedding_loss_repulsive=   0.00030
[04:21:45 TCNTrainer] DEBUG: Epoch 8 (  160/203): Total=   0.00267, embedding_loss_attractive=   0.00215, embedding_loss_repulsive=   0.00029
[04:21:46 TCNTrainer] DEBUG: Epoch 8 (  170/203): Total=   0.00257, embedding_loss_attractive=   0.00223, embedding_loss_repulsive=   0.00023
[04:21:46 TCNTrainer] DEBUG: Epoch 8 (  180/203): Total=   0.00263, embedding_loss_attractive=   0.00222, embedding_loss_repulsive=   0.00025
[04:21

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:21:51 TCNTrainer] DEBUG: Epoch 10 (    0/203): Total=   0.00271, embedding_loss_attractive=   0.00225, embedding_loss_repulsive=   0.00025
[04:21:51 TCNTrainer] DEBUG: Epoch 10 (   10/203): Total=   0.00261, embedding_loss_attractive=   0.00230, embedding_loss_repulsive=   0.00020
[04:21:52 TCNTrainer] DEBUG: Epoch 10 (   20/203): Total=   0.00265, embedding_loss_attractive=   0.00222, embedding_loss_repulsive=   0.00024
[04:21:52 TCNTrainer] DEBUG: Epoch 10 (   30/203): Total=   0.00281, embedding_loss_attractive=   0.00244, embedding_loss_repulsive= 

In [23]:
constraints_2 = [(0.0009653154573041118, 5.0, 1.0, 1.0),
               (0.0004771597001106582, 5.0, 1.0, 1.0),
               (0.00028893887271952276, 5.0, 1.0, 1.0),
               (0.00016839923191582784, 5.0, 1.0, 1.0),
               (0.00010062895363475553, 5.0, 1.0, 1.0)]

In [24]:
constraints_3 = [(0.0009653154573041118, 1.0, 1.0, 1.0),
               (0.0004771597001106582, 1.0, 1.0, 1.0),
               (0.00028893887271952276, 1.0, 1.0, 1.0),
               (0.00016839923191582784, 1.0, 1.0, 1.0),
               (0.00010062895363475553, 1.0, 1.0, 1.0)]

In [25]:
save_loss_history(constraints_2, "loss_histories/damping_05")
save_loss_history(constraints_3, "loss_histories/damping_01")

[04:26:23 TCNTrainer] INFO: Using device cuda


Training for = (0.0009653154573041118, 5.0, 1.0, 1.0)


/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:26:23 TCNTrainer] DEBUG: Epoch 1 (    0/203): Total=   0.03381, embedding_loss_attractive=   0.03381, embedding_loss_repulsive=   0.00107
[04:26:23 TCNTrainer] DEBUG: Epoch 1 (   10/203): Total=   0.00458, embedding_loss_attractive=   0.00425, embedding_loss_repulsive=   0.00656
[04:26:24 TCNTrainer] DEBUG: Epoch 1 (   20/203): Total=   0.00302, embedding_loss_attractive=   0.00256, embedding_loss_repulsive=   0.00527
[04:26:24 TCNTrainer] DEBUG: Epoch 1 (   30/203): Total=   0.00263, embedding_loss_attractive=   0.00210, embedding_loss_repulsive=   0.

[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  120/203): Total=   0.00191, embedding_loss_attractive=   0.00187, embedding_loss_repulsive=   0.00104
[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  130/203): Total=   0.00180, embedding_loss_attractive=   0.00184, embedding_loss_repulsive=   0.00090
[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  140/203): Total=   0.00182, embedding_loss_attractive=   0.00180, embedding_loss_repulsive=   0.00100
[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  150/203): Total=   0.00175, embedding_loss_attractive=   0.00172, embedding_loss_repulsive=   0.00101
[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  160/203): Total=   0.00173, embedding_loss_attractive=   0.00169, embedding_loss_repulsive=   0.00104
[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  170/203): Total=   0.00166, embedding_loss_attractive=   0.00161, embedding_loss_repulsive=   0.00105
[04:26:30 TCNTrainer] DEBUG: Epoch 2 (  180/203): Total=   0.00162, embedding_loss_attractive=   0.00169, embedding_loss_repulsive=   0.00084
[04:26

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:26:36 TCNTrainer] DEBUG: Epoch 4 (    0/203): Total=   0.00152, embedding_loss_attractive=   0.00147, embedding_loss_repulsive=   0.00105
[04:26:36 TCNTrainer] DEBUG: Epoch 4 (   10/203): Total=   0.00144, embedding_loss_attractive=   0.00145, embedding_loss_repulsive=   0.00096
[04:26:36 TCNTrainer] DEBUG: Epoch 4 (   20/203): Total=   0.00140, embedding_loss_attractive=   0.00149, embedding_loss_repulsive=   0.00082
[04:26:37 TCNTrainer] DEBUG: Epoch 4 (   30/203): Total=   0.00140, embedding_loss_attractive=   0.00148, embedding_loss_repulsive=   0.

...

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:32:57 TCNTrainer] DEBUG: Epoch 4 (    0/203): Total=   0.00276, embedding_loss_attractive=   0.00206, embedding_loss_repulsive=   0.00065
[04:32:57 TCNTrainer] DEBUG: Epoch 4 (   10/203): Total=   0.00299, embedding_loss_attractive=   0.00223, embedding_loss_repulsive=   0.00069
[04:32:57 TCNTrainer] DEBUG: Epoch 4 (   20/203): Total=   0.00295, embedding_loss_attractive=   0.00243, embedding_loss_repulsive=   0.00049
[04:32:58 TCNTrainer] DEBUG: Epoch 4 (   30/203): Total=   0.00290, embedding_loss_attractive=   0.00230, embedding_loss_repulsive=   0.

[04:33:03 TCNTrainer] DEBUG: Epoch 5 (  120/203): Total=   0.00294, embedding_loss_attractive=   0.00235, embedding_loss_repulsive=   0.00044
[04:33:03 TCNTrainer] DEBUG: Epoch 5 (  130/203): Total=   0.00282, embedding_loss_attractive=   0.00219, embedding_loss_repulsive=   0.00046
[04:33:04 TCNTrainer] DEBUG: Epoch 5 (  140/203): Total=   0.00278, embedding_loss_attractive=   0.00218, embedding_loss_repulsive=   0.00044
[04:33:04 TCNTrainer] DEBUG: Epoch 5 (  150/203): Total=   0.00278, embedding_loss_attractive=   0.00215, embedding_loss_repulsive=   0.00045
[04:33:04 TCNTrainer] DEBUG: Epoch 5 (  160/203): Total=   0.00293, embedding_loss_attractive=   0.00236, embedding_loss_repulsive=   0.00042
[04:33:04 TCNTrainer] DEBUG: Epoch 5 (  170/203): Total=   0.00283, embedding_loss_attractive=   0.00217, embedding_loss_repulsive=   0.00047
[04:33:04 TCNTrainer] DEBUG: Epoch 5 (  180/203): Total=   0.00277, embedding_loss_attractive=   0.00223, embedding_loss_repulsive=   0.00039
[04:33

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:33:10 TCNTrainer] DEBUG: Epoch 7 (    0/203): Total=   0.00259, embedding_loss_attractive=   0.00213, embedding_loss_repulsive=   0.00032
[04:33:10 TCNTrainer] DEBUG: Epoch 7 (   10/203): Total=   0.00251, embedding_loss_attractive=   0.00207, embedding_loss_repulsive=   0.00031
[04:33:10 TCNTrainer] DEBUG: Epoch 7 (   20/203): Total=   0.00264, embedding_loss_attractive=   0.00217, embedding_loss_repulsive=   0.00032
[04:33:10 TCNTrainer] DEBUG: Epoch 7 (   30/203): Total=   0.00270, embedding_loss_attractive=   0.00206, embedding_loss_repulsive=   0.

[04:33:16 TCNTrainer] DEBUG: Epoch 8 (  120/203): Total=   0.00250, embedding_loss_attractive=   0.00211, embedding_loss_repulsive=   0.00026
[04:33:16 TCNTrainer] DEBUG: Epoch 8 (  130/203): Total=   0.00255, embedding_loss_attractive=   0.00206, embedding_loss_repulsive=   0.00030
[04:33:16 TCNTrainer] DEBUG: Epoch 8 (  140/203): Total=   0.00266, embedding_loss_attractive=   0.00213, embedding_loss_repulsive=   0.00031
[04:33:16 TCNTrainer] DEBUG: Epoch 8 (  150/203): Total=   0.00263, embedding_loss_attractive=   0.00215, embedding_loss_repulsive=   0.00029
[04:33:17 TCNTrainer] DEBUG: Epoch 8 (  160/203): Total=   0.00276, embedding_loss_attractive=   0.00222, embedding_loss_repulsive=   0.00032
[04:33:17 TCNTrainer] DEBUG: Epoch 8 (  170/203): Total=   0.00251, embedding_loss_attractive=   0.00198, embedding_loss_repulsive=   0.00031
[04:33:17 TCNTrainer] DEBUG: Epoch 8 (  180/203): Total=   0.00253, embedding_loss_attractive=   0.00220, embedding_loss_repulsive=   0.00023
[04:33

/scratch/gpfs/dc3896/micromamba/envs/gnn/lib/python3.10/site-packages/torch_geometric/data/collate.py:145: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  storage = elem.storage()._new_shared(numel)
[04:33:23 TCNTrainer] DEBUG: Epoch 10 (    0/203): Total=   0.00273, embedding_loss_attractive=   0.00221, embedding_loss_repulsive=   0.00029
[04:33:23 TCNTrainer] DEBUG: Epoch 10 (   10/203): Total=   0.00262, embedding_loss_attractive=   0.00219, embedding_loss_repulsive=   0.00026
[04:33:23 TCNTrainer] DEBUG: Epoch 10 (   20/203): Total=   0.00259, embedding_loss_attractive=   0.00209, embedding_loss_repulsive=   0.00028
[04:33:23 TCNTrainer] DEBUG: Epoch 10 (   30/203): Total=   0.00258, embedding_loss_attractive=   0.00216, embedding_loss_repulsive= 